In [1]:
import pydot

In [2]:
import sys

In [3]:
import gi

In [4]:
gi.require_version('Gst', '1.0')

In [5]:
gi.require_version("GstApp", "1.0")

In [6]:
gi.require_version('GstVideo', '1.0')

In [7]:
gi.require_version('GstRtspServer', '1.0')
gi.require_version('GIRepository', '2.0')

In [8]:
from gi.repository import GObject, Gst, GstVideo, GstRtspServer, GLib, GIRepository

In [9]:
Gst.init(None)

[]

In [10]:
mainloop = GLib.MainLoop()

In [11]:
server = GstRtspServer.RTSPServer.new()
server.props.service = "5000"
server.attach(None)

1

In [12]:
mounts = server.get_mount_points()

In [13]:
factory = GstRtspServer.RTSPMediaFactory()

In [14]:
factory.set_launch('(mediasrcbin name=videosrc media-device=/dev/media1  v4l2src0::io-mode=dmbuf-import \
! video/x-raw, width=1920, height=1080, format=NV12, framerate=30/1  \
! tee name=t  ! queue ! ivas_xm2m kconfig="/home/root/somapp/kernel_xpp_pipeline.json" \
! video/x-raw, width=640, height=360, format=BGR \
! queue ! ivas_xfilter kernels-config="/home/root/somapp/kernel_densebox_640_360.json" \
! ima.sink_master ivas_xmetaaffixer name=ima ima.src_master ! fakesink \
t.  ! ima.sink_slave_0 ima.src_slave_0 ! queue \
! ivas_xfilter kernels-config="/home/root/somapp/kernel_boundingbox_facedetect.json" \
! queue ! ivas_xroigen roi-type=1 roi-qp-delta=-10 roi-max-num=10   \
! queue ! omxh265enc qp-mode=auto num-slices=8 gop-length=60 periodicity-idr=270 control-rate=low-latency \
  gop-mode=low-delay-p gdr-mode=horizontal cpb-size=200 initial-delay=100  filler-data=false min-qp=15 \
  max-qp=40  b-frames=0  low-bandwidth=false  \
! video/x-h265, alignment=au \
! queue ! rtph265pay name=pay0 pt=96 )')

factory.set_shared(True)

In [15]:
mounts.add_factory("/test", factory)

In [16]:
print ("stream ready at rtsp://127.0.0.1:{}/test".format(server.props.service))

stream ready at rtsp://127.0.0.1:5000/test


In [17]:
mainloop.run()